In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
import xarray as xr

# Open dataset with cfgrib
dat = xr.open_dataset('ERA5/htr_00_10.grib', engine='cfgrib')

# Add a monthly timestamp to use for grouping
dat.coords['month_period'] = ('time', dat['time'].to_index().to_period('M').to_timestamp())

# Initialize output list
H = []

# Loop over latitude bands: 80–90, 70–80, 60–70
for i in range(3):
    lat_min = 80 - 10 * i
    lat_max = 90 - 10 * i
    
    # Select latitudes in this band
    sub = dat.where((dat.latitude > lat_min) & (dat.latitude <= lat_max), drop=True)

    # Average over longitude (or 'values' if cfgrib named it oddly)
    if 'longitude' in sub.dims:
        zonal_mean = sub['avg_ttpm'].mean(dim='longitude')
    elif 'values' in sub.dims:
        zonal_mean = sub['avg_ttpm'].mean(dim='values')
    else:
        raise ValueError("No longitude or values dimension found.")
    
    # Group by monthly period and average
    monthly_mean = zonal_mean.groupby('month_period').mean(dim='time')
    
    H.append(monthly_mean)


In [ ]:
dat.groupby('month_period').mean(dim='values')

In [5]:
H[0]*86400

<xarray.DataArray 'avg_ttlwr' (month_period: 120, hybrid: 137)> Size: 66kB
array([[ -8.028087 ,  -5.270738 ,  -6.781654 , ...,  -1.5786881,
         -2.3192413,  -2.6691797],
       [ -8.754998 ,  -5.432702 ,  -6.1558495, ...,  -1.7170894,
         -2.6991751,  -3.8528762],
       [ -6.17731  ,  -3.8587565,  -4.723885 , ...,  -1.2463226,
         -1.98207  ,  -2.717963 ],
       ...,
       [ -6.4374194,  -5.1452174,  -7.084013 , ...,  -1.4456515,
         -1.73473  ,  -2.168367 ],
       [-11.400361 ,  -9.72708  , -12.00772  , ...,  -1.798319 ,
         -2.3238678,  -2.426485 ],
       [ -9.930902 ,  -8.182856 , -10.485174 , ...,  -1.4348972,
         -2.117824 ,  -2.494117 ]], dtype=float32)
Coordinates:
    step          timedelta64[ns] 8B 12:00:00
  * hybrid        (hybrid) float64 1kB 1.0 2.0 3.0 4.0 ... 135.0 136.0 137.0
  * month_period  (month_period) datetime64[ns] 960B 1999-12-01 ... 2009-11-01

In [4]:
for i in range(3):
    H[i].to_netcdf('ERA5/htrlw'+str(90-10*i))
    #S[i].to_netcdf('Sol'+str(90-10*i))